In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
cd /content/drive/My Drive/FastCampus

/content/drive/My Drive/FastCampus


# <center>__컨텐츠 팀__</center>
# <center>__결과 보고서__</center>



##__1. 프로젝트 수행 목적__
###동기  
  + 개봉 예정 영화에 대한 흥행 정도에 대한 호기심  

###목적
  + 개봉 예정된 영화에 대한 흥행 정도를 기대지수, 감복 및 배우의 수상 경력 등의 지표로 나누어
  수치화.



##__2. 데이터 수집__

 ### 개봉 예정 영화 데이터 수집

<img src="naver_movie.jpg"></img>
![Alt text](./naver_movie.jpg)

 

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# 네이버 영화 url
url = "https://movie.naver.com/movie/running/premovie.nhn"
# requests 요청
response = requests.get(url)
# 응답 확인
response
# 해당 text를 html로 변환
html = BeautifulSoup(response.text,"html.parser")
# 특정 태그 선택
data = html.select('div.obj_section div.lst_wrap ul.lst_detail_t1 li')
                        # 설명              Column Name
                        # 
names = []              # 영화 제목         Name Actor
directors = []          # 영화 감독명       Director
actors = []             # 영화 배우명       Actor
ratings = []            # 영화 관람등급     Rating
genres = []             # 영화 장르         Genre
times = []              # 영화 상영시간     Time
release_dates = []      # 영화 개봉예정일   Release Date
anticipate_up = []      # 기대지수 UP       anticipate Up
anticipate_down = []    # 기대지수 DOWN     anticipate Down

for item in data:
    # 각 컬럼별 태그 위치를 찾아서 리스트에 추가, 없다면 결측치 입력
    try:
        names.append(item.select('dl.lst_dsc dt.tit a')[0].text.strip())
    except IndexError:
        names.append(None)
    try:
        directors.append(re.sub('[\r\t\n]','',item.select('dl.lst_dsc dl.info_txt1 dd')[1].text.strip()))
    except IndexError:
        directors.append(None)
    try:
        actors.append(re.sub('[\r\t\n]','',item.select('dl.lst_dsc dl.info_txt1 dd')[2].text.strip()))
    except IndexError:
        actors.append(None)
    try:
        ratings.append(item.select('dl.lst_dsc dt.tit span')[0].text.strip())
    except IndexError:
        ratings.append(None)
    try:
        genre = item.select('dl.lst_dsc dl.info_txt1 dd')[0].select('a')[0].text.strip()
        if len(item.select('dl.lst_dsc dl.info_txt1 dd')[0].select('a')) > 1:
            for i in item.select('dl.lst_dsc dl.info_txt1 dd')[0].select('a')[1:]:
                genre += f', {i.text.strip()}'
        genres.append(genre)
    except IndexError:
        genres.append(None)
    try:
        star =  item.select('dl.lst_dsc dd.star dl.info_exp em')
        anticipate_up.append(star[0].text.strip())
        anticipate_down.append(star[1].text.strip())
    except IndexError:
        anticipate_up.append(None)
        anticipate_down.append(None)
    instance = re.sub('[\r\t\n]','',item.select('dl.info_txt1 dd')[0].text.strip()).split('|')
    tflag = False
    rflag = False
    for i in instance:
        if '분' in i:
            times.append(i.replace("분",""))
            tflag = True
        elif '개봉' in i:
            release_dates.append(i.replace("개봉","").strip())
            rflag = True
        else :
            continue
    if tflag == False :
        times.append(None)
    if rflag == False:
        release_dates.append(None)
# 데이터 프레임 생성
df = pd.DataFrame(columns=['Name','Director','Actor','Rating','Genre','Time','Release Date'])
df['Name'] = names
df['Director'] = directors
df['Actor'] = actors
df['Rating'] = ratings
df['Genre'] = genres
df['Time'] = times
df['Release Date'] = release_dates
df['anticipate Up'] = anticipate_up
df['anticipate Down'] = anticipate_down
# 기대지수가 없는 경우(재개봉)를 제외한 새로운 데이터프레임 생성
df2 = df.dropna(subset=['anticipate Up'])
# 인덱스 재지정 및 삭제
df2.reset_index(inplace=True)
del df2['index']

df.to_csv('./movie_final.csv',encoding='utf-8-sig')
df2.to_csv('./movies_2.csv',encoding='utf-8-sig')

In [ ]:
df2

,Name,Director,Actor,Rating,Genre,Time,Release Date,anticipate Up,anticipate Down
0,데스 오브 미,대런 린 보우즈만,"매기 큐, 루크 헴스워스",청소년 관람불가,"공포, 스릴러",94,2021.02.08,18,3
1,새해전야,홍지영,"김강우, 유인나, 유연석, 이연희, 이동휘, 천두링, 염혜란, 수영, 유태오",12세 관람가,멜로/로맨스,114,2021.02.10,6808,439
2,몬스터 헌터,폴 앤더슨,"밀라 요보비치, 토니 자",12세 관람가,"액션, 모험",103,2021.02.10,396,46
3,아이,김현탁,"김향기, 류현경, 염혜란",15세 관람가,드라마,113,2021.02.10,852,81
4,살아남은 사람들,버르너바시 토트,"카롤리 하이덕, 마리 나기, 아비겔 소크",15세 관람가,"드라마, 멜로/로맨스",88,2021.02.10,61,1
...,...,...,...,...,...,...,...,...,...
60,피넛 버터 팔콘,"타일러 닐슨, 마이크 슈왈츠","샤이아 라보프, 다코타 존슨, 잭 고츠아전, 존 호키스",12세 관람가,드라마,97,2021.03,37,4
61,추룡 2 : 패왕,"왕정, 관지요","양가휘, 고천락, 임가동, 임달화",15세 관람가,"액션, 범죄, 스릴러",101,2021.04.08,5,3
62,분노의 질주: 더 얼티메이트,저스틴 린,"빈 디젤, 존 시나, 샤를리즈 테론, 미셸 로드리게즈, 조다나 브류스터",None,액션,None,2021.05,10697,296
63,007 노 타임 투 다이,캐리 후쿠나가,"다니엘 크레이그, 라미 말렉, 라샤나 린치, 레아 세이두",None,액션,None,2021.10,5237,188


##__3. 데이터 탐색 및 분석__

In [ ]:
import csv
 
f = open('rrr.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    print(line)
f.close()


['movie', 'award', 'award_name']
['분노의 질주: 더 얼티메이트', '후보', '제 43회 새턴 어워즈 (최우수 SF영화상)']
['분노의 질주: 더 얼티메이트', '후보', '제 22회 크리틱스 초이스 시상식 (SF/호러영화상)']
['분노의 질주: 더 얼티메이트', '후보', '제 33회 로스앤젤레스 아시안 퍼시픽 영화제 (갈라 프레젠테이션)']
['분노의 질주: 더 얼티메이트', '후보', '제 33회 로스앤젤레스 아시안 퍼시픽 영화제 (스페셜 프레젠테이션)']
['분노의 질주: 더 얼티메이트', '후보', '제 18회 선댄스영화제 (심사위원대상(미국 드라마))']
['분노의 질주: 더 얼티메이트', '후보', '제 34회 샌프란시스코 국제 아시안 아메리칸 영화제 (스페셜 프레젠테이션)']
['분노의 질주: 더 얼티메이트', '수상', '제 40회 새턴 어워즈 (최우수 액션,모험상)']
['분노의 질주: 더 얼티메이트', '후보', '제 38회 새턴 어워즈 (최우수 액션,모험상)']
['분노의 질주: 더 얼티메이트', '후보', '제 18회 MTV 영화 & TV 어워즈 (최고의 영화상)']
['분노의 질주: 더 얼티메이트', '후보', '제 27회 하와이국제영화제 (American Immigrant Filmmakers)']
['분노의 질주: 더 얼티메이트', '후보', '제 23회 선댄스영화제 (미드나잇)']
['분노의 질주: 더 얼티메이트', '후보', '제 25회 MTV 영화 & TV 어워즈 (최고의 액션연기상)']
['분노의 질주: 더 얼티메이트', '수상', '제 23회 MTV 영화 & TV 어워즈 (최고의 콤비상)']
['분노의 질주: 더 얼티메이트', '후보', '제 18회 MTV 영화 & TV 어워즈 (최고의 남자배우상)']
['분노의 질주: 더 얼티메이트', '후보', '제 25회 골든 라즈베리 시상식 (최악의 남우주연상)']
['분노의 질주: 더 얼티메이트', '후보', '제 12회 MTV 영화 & TV 어워즈 